# Installation Procedure 
Before we start, the scripts written to install PandExo assume the user has installed [Anaconda](https://www.continuum.io/downloads). For diehard pip users, know that STScI tools will be only distributed on conda so PandExo will not be the only package to use it. 

If you are dead set against it you will need to follow the installation guidelines [here](https://natashabatalha.github.io/PandExo/installation.html). 

If you have conda then, let's proceed below. 

## Download Data

As mentioned, the user must provide their own reference data files in order for the engine to understand the instrument with which the calculations will be performed. First step to getting PandExo running is downloading four files. Download them and put them in the same directory. 

#### JWST Reference Data
For JWST, the user may download the required reference data [here](http://ssb.stsci.edu/pandeia/engine/1.0/pandeia_data-1.0.tar.gz). Unpack and store the data in a choice location.

#### Stellar SEDs 
Likewise, the user may wish to install specific data for use with the PySynPhot package Pandeia uses. We will only be using the Phoenix stellar atlas, which can be downloaded [here](ftp://ftp.stsci.edu/cdbs/tarfiles/synphot5.tar.gz). 

#### PandExo startup bash script
A startup script and a python test script was created to do an automatic install of PandExo. You can download both of them from a google drive located [here](https://drive.google.com/open?id=0ByRvbM3zW0d-d2xoSmNMc1RtdTQ).


## Running Setup Script 

Open up the file `pandexo.sh`. There are two lines in the beginning which tell PandExo where your reference data will go. Currently they will unpack and store the reference files in the HOME directory. 

If you are not okay with it, **change** $HOME to whatever you want the data to be.

The run

```bash
sh pandexo.sh
```

Hopefully you have had success but if not here, let's try and trouble shoot. 

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandexo.engine.justdoit as jdi # THIS IS THE HOLY GRAIL OF PANDEXO
import numpy as np
import os
#pip install pandexo.engine --upgrade

# Running PandExo 


## Editting Inputs

### Step 1) Load in a blank exoplanet dictionary

To start, load in a blank exoplanet dictionary with empty keys. You will fill these out for yourself in the next step. 

In [3]:
exo_dict = jdi.load_exo_dict()
print(exo_dict.keys())
#print(exo_dict['star']['w_unit'])

dict_keys(['calculation', 'star', 'planet', 'observation'])


### Edit exoplanet observation inputs
Editting each keys are annoying. But, do this carefully or it could result in nonsense runs

In [4]:
exo_dict['observation']['sat_level'] = 80    #saturation level in percent of full well 
exo_dict['observation']['sat_unit'] = '%'
exo_dict['observation']['noccultations'] = 2 #number of transits 
exo_dict['observation']['R'] = None          #fixed binning. I usually suggest ZERO binning.. you can always bin later 
                                             #without having to redo the calcualtion
exo_dict['observation']['baseline_unit'] = 'total'  #Defines how you specify out of transit observing time
                                                    #'frac' : fraction of time in transit versus out = in/out 
                                                    #'total' : total observing time (seconds)
exo_dict['observation']['baseline'] = 4.0*60.0*60.0 #in accordance with what was specified above (total observing time)

exo_dict['observation']['noise_floor'] = 0   #this can be a fixed level or it can be a filepath 
                                             #to a wavelength dependent noise floor solution (units are ppm)

### Edit exoplanet host star inputs
Note... If you select 'phoenix' you **do not** have to provide a starpath, w_unit or f_unit, but you **do** have to provide a temp, metal and logg. If you select 'user' you **do not** need to provide a temp, metal and logg, but you **do** need to provide units and starpath. 

In [5]:
exo_dict['star']['type'] = 'phoenix'        #phoenix or user (if you have your own)
exo_dict['star']['mag'] = 8.0               #magnitude of the system
exo_dict['star']['ref_wave'] = 1.25         #For J mag = 1.25, H = 1.6, K =2.22.. etc (all in micron)
exo_dict['star']['temp'] = 5500             #in K 
exo_dict['star']['metal'] = 0.0             # as log Fe/H
exo_dict['star']['logg'] = 4.0              #log surface gravity cgs

### Edit exoplanet inputs using one of  three options: 1) user specified, 2) constant value, 3) select from grid

#### 1) Edit exoplanet planet inputs if using your own model

In [ ]:
exo_dict['planet']['type'] ='user'                       #tells pandexo you are uploading your own spectrum
exo_dict['planet']['exopath'] = 'wasp12b.txt'
exo_dict['planet']['w_unit'] = 'cm'                      #other options include "um","nm" ,"Angs", "sec" (for phase curves)
exo_dict['planet']['f_unit'] = 'rp^2/r*^2'               #other options are 'fp/f*' 
exo_dict['planet']['transit_duration'] = 2.0*60.0*60.0   #transit duration 
exo_dict['planet']['td_unit'] = 's'                      #Any unit of time in accordance with astropy.units can be added

#### 2) Users can also add in a constant temperature or a constant transit depth 

In [ ]:
exo_dict['planet']['type'] = 'constant'                  #tells pandexo you want a fixed transit depth
exo_dict['planet']['transit_duration'] = 2.0*60.0*60.0   #transit duration 
exo_dict['planet']['td_unit'] = 's' 
exo_dict['planet']['radius'] = 1
exo_dict['planet']['r_unit'] = 'R_jup'            #Any unit of distance in accordance with astropy.units can be added here
exo_dict['star']['radius'] = 1
exo_dict['star']['r_unit'] = 'R_sun'              #Same deal with astropy.units here
exo_dict['planet']['f_unit'] = 'rp^2/r*^2'        #this is what you would do for primary transit 

#ORRRRR....
#if you wanted to instead to secondary transit at constant temperature 
exo_dict['planet']['f_unit'] = 'fp/f*' 
exo_dict['planet']['temp'] = 1000

#### 3) Select from grid
NOTE: Currently only the fortney grid for hot Jupiters from Fortney+2010 is supported. Holler though, if you want another grid supported

In [10]:
exo_dict['planet']['type'] = 'grid'                #tells pandexo you want to pull from the grid
exo_dict['planet']['temp'] = 1000                 #grid: 500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500
exo_dict['planet']['chem'] = 'noTiO'              #options: 'noTiO' and 'eqchem', noTiO is chemical eq. without TiO
exo_dict['planet']['cloud'] = 'ray10'               #options: nothing: '0', 
#                                                   Weak, medium, strong scattering: ray10,ray100, ray1000
#                                                   Weak, medium, strong cloud: flat1,flat10, flat100
exo_dict['planet']['mass'] = 1
exo_dict['planet']['m_unit'] = 'M_jup'            #Any unit of mass in accordance with astropy.units can be added here
exo_dict['planet']['radius'] = 1
exo_dict['planet']['r_unit'] = 'R_jup'            #Any unit of distance in accordance with astropy.units can be added here
exo_dict['star']['radius'] = 1
exo_dict['star']['r_unit'] = 'R_sun'              #Same deal with astropy.units here
exo_dict['planet']['transit_duration'] = 2.0*60.0*60.0   #transit duration 
exo_dict['planet']['td_unit'] = 's' 

### Step 2) Load in instrument dictionary (OPTIONAL)
 
Step 2 is optional because PandExo has the functionality to automatically load in instrument dictionaries. Skip this if you plan on observing with one of the following and want to use the subarray with the smallest frame time and the readout mode with 1 frame/1 group (standard): 
- NIRCam F444W
- NIRSpec Prism
- NIRSpec G395M
- NIRSpec G395H
- NIRSpec G235H
- NIRSpec G235M
- NIRCam F322W
- NIRSpec G140M
- NIRSpec G140H
- MIRI LRS
- NIRISS SOSS

In [ ]:
#jdi.print_instruments()
result = jdi.run_pandexo(exo_dict,['NIRCam F322W2'])

Running Single Case for: NIRCam F322W2
Optimization Reqested: Computing Duty Cycle


In [ ]:
inst_dict = jdi.load_mode_dict('NIRSpec G140H')

#loading in instrument dictionaries allow you to personalize some of  
#the fields that are predefined in the templates. The templates have 
#the subbarays with the lowest frame times and the readmodes with 1 frame per group. 
#if that is not what you want. change these fields

#Try printing this out to get a feel for how it is structured: 

print(inst_dict['configuration'])

In [ ]:
#Another way to display this is to print out the keys 
inst_dict.keys()

#### Dont know what field to choose?? Print them out! 

In [ ]:
print("SUBARRAYS")
print(jdi.subarrays('nirspec'))

print("FILTERS")
print(jdi.filters('nircam'))

print("DISPERSERS")
print(jdi.dispersers('nirspec'))

In [ ]:
#you can try personalizing some of these fields

inst_dict["configuration"]["detector"]["ngroup"] = 'optimize'   #running "optimize" will select the maximum 
                                                                #possible groups before saturation. 
                                                                #You can also write in any integer between 2-65536

inst_dict["configuration"]["detector"]["subarray"] = 'substrip256'   #change the subbaray



## Running PandExo

You have **four options** for running PandExo. All of them are accessed through attribute **jdi.run_pandexo**. See examples below. 

`    jdi.run_pandexo(exo, inst, param_space = 0, param_range = 0,save_file = True,
                            output_path=os.getcwd(), output_file = '')`



### Option 1- Run single instrument mode, single planet
If you forget which instruments are available run **jdi.print_isntruments()** and pick one 

In [ ]:
jdi.print_instruments()

In [ ]:
result = jdi.run_pandexo(exo_dict,['NIRCam F322W2'])

### Option 2- Run single instrument mode (with user dict), single planet
This is the same thing as option 1 but instead of feeding it a list of keys, you can feed it a instrument dictionary (this is for users who wanted to simulate something NOT pre defined within pandexo)

In [ ]:
inst_dict = jdi.load_mode_dict('NIRSpec G140H')
#personalize subarray
inst_dict["configuration"]["detector"]["subarray"] = 'sub2048'
result = jdi.run_pandexo(exo_dict, inst_dict)

### Option 3- Run several modes, single planet
Use several modes from **print_isntruments()** options. 

In [ ]:
#choose select 
result = jdi.run_pandexo(exo_dict,['NIRSpec G140M','NIRSpec G235M','NIRSpec G395M'],
               output_file='three_nirspec_modes.p')
#run all 
#result = jdi.run_pandexo(exo_dict, ['RUN ALL'], save_file = False)

### Option 4- Run single mode, several planet cases
Use a single modes from **print_isntruments()** options. But explore parameter space with respect to **any** parameter in the exo dict. The example below shows how to loop over several planet models

You can loop through anything in the exoplanet dictionary. It will be planet, star or observation followed by whatever you want to loop through in that set. 

i.e. planet+exopath, star+temp, star+metal, star+logg, observation+sat_level.. etc

In [ ]:
#looping over different exoplanet models 
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'planet+exopath',
                param_range = os.listdir('/path/to/location/of/models'),
               output_path = '/path/to/output/simulations')

#looping over different stellar temperatures 
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'star+temp',
                param_range = np.linspace(5000,8000,2),
               output_path = '/path/to/output/simulations')

#looping over different saturation levels
jdi.run_pandexo(exo_dict, ['NIRCam F444W'], param_space = 'observation+sat_level',
                param_range = np.linspace(.5,1,5),
               output_path = '/path/to/output/simulations')
